In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

In [ ]:
raw_data = pd.read_csv('data.csv').drop(['Unnamed: 0'], axis=1)

In [3]:
raw_data

,FIT101_0,LIT101_0,MV101_0,P101_0,P102_0,AIT201_0,AIT202_0,AIT203_0,FIT201_0,MV201_0,...,FIT504_3,P501_3,P502_3,PIT501_3,PIT502_3,PIT503_3,FIT601_3,P601_3,P602_3,P603_3
0,0.001538,0.331411,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000196,0.001268,0.000634,0.158997,0.001046,0.120169,8.121678e-08,0.000634,0.000634,0.000634
1,0.001551,0.331436,0.001268,0.001268,0.000634,0.166081,0.005322,0.208307,0.001550,0.001268,...,0.000196,0.001267,0.000634,0.158940,0.001045,0.120055,8.118763e-08,0.000634,0.000634,0.000634
2,0.001578,0.331392,0.001268,0.001268,0.000634,0.166072,0.005321,0.208295,0.001548,0.001268,...,0.000195,0.001267,0.000633,0.158818,0.001045,0.120025,8.116716e-08,0.000633,0.000633,0.000633
3,0.001606,0.331431,0.001268,0.001268,0.000634,0.166054,0.005320,0.208273,0.001548,0.001268,...,0.000195,0.001267,0.000633,0.158715,0.001045,0.120023,8.116590e-08,0.000633,0.000633,0.000633
4,0.001628,0.331635,0.001267,0.001267,0.000634,0.165994,0.005318,0.208198,0.001548,0.001267,...,0.000195,0.001266,0.000633,0.158673,0.001045,0.119991,8.114453e-08,0.000633,0.000633,0.000633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449907,0.001691,0.327946,0.001270,0.001270,0.000635,0.106759,0.005489,0.191702,0.001564,0.001270,...,0.000195,0.001270,0.000635,0.159641,0.000600,0.120102,0.000000e+00,0.000635,0.000635,0.000635
449908,0.001690,0.328063,0.001270,0.001270,0.000635,0.106756,0.005489,0.191697,0.001564,0.001270,...,0.000195,0.001270,0.000635,0.159628,0.000600,0.120093,0.000000e+00,0.000635,0.000635,0.000635
449909,0.001673,0.328205,0.001270,0.001270,0.000635,0.106754,0.005489,0.191693,0.001564,0.001270,...,0.000195,0.001270,0.000635,0.159586,0.000600,0.120061,0.000000e+00,0.000635,0.000635,0.000635
449910,0.001668,0.328303,0.001270,0.001270,0.000635,0.106753,0.005489,0.191692,0.001563,0.001270,...,0.000195,0.001269,0.000635,0.159556,0.000549,0.120038,0.000000e+00,0.000635,0.000635,0.000635


In [ ]:
with open('labels_for_windowed_swat.pickle', 'rb') as f:
    y = pickle.load(f)[:449912]

In [ ]:
x_train = raw_data[y == 0.0][:200_000]

In [ ]:
train_test_split = x_train.index[-1]

In [ ]:
x_test = raw_data[train_test_split:]

In [ ]:
y_test = y[train_test_split:]

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [ ]:
x_train_encoder = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test_encoder = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [ ]:
LSTM_AE = Sequential()
LSTM_AE.add(LSTM(204, return_sequences=True, activation='relu', input_shape=(x_train_encoder.shape[1], x_train_encoder.shape[2])))
LSTM_AE.add(LSTM(102, return_sequences=True, activation='relu'))
LSTM_AE.add(LSTM(102, return_sequences=True, activation='relu'))
LSTM_AE.add(LSTM(204, return_sequences=True, activation='relu'))
LSTM_AE.add(Dense(x_train_encoder.shape[2]))
LSTM_AE.compile(optimizer='adam', loss='mae')
LSTM_AE.summary()

In [43]:
history = LSTM_AE.fit(x_train_encoder,
                      x_train_encoder,
                      epochs=10,
                      batch_size=100,
                      shuffle=False,
                      verbose=1)

Epoch 1/10
2000/2000 [==============================] - 1230s 614ms/step - loss: 0.0489
Epoch 2/10
2000/2000 [==============================] - 1229s 614ms/step - loss: 0.0172
Epoch 3/10
2000/2000 [==============================] - 1226s 613ms/step - loss: 0.0015
Epoch 4/10
2000/2000 [==============================] - 1222s 611ms/step - loss: 0.0014
Epoch 5/10
 127/2000 [>.............................] - ETA: 19:07 - loss: 0.0016

KeyboardInterrupt: 

In [ ]:
x_train_pred = LSTM_AE.predict(x_train_encoder)

In [ ]:
train_mae_loss = np.mean(np.abs(x_train_pred - x_train_encoder), axis=1)

In [ ]:
threshold = np.max(train_mae_loss)threshold

In [48]:
threshold

0.005552887733512842

In [ ]:
x_test_pred = LSTM_AE.predict(x_test_encoder)

In [ ]:
test_mae_loss = np.mean(np.abs(x_test_pred - x_test_encoder), axis=1)
test_mae_loss = test_mae_loss.reshape((-1))

In [ ]:
anomalies_encoder = test_mae_loss > threshold

In [55]:
print(classification_report(y[train_test_split:], anomalies_encoder))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96    195187
           1       0.83      0.80      0.82     42906

    accuracy                           0.93    238093
   macro avg       0.89      0.88      0.89    238093
weighted avg       0.93      0.93      0.93    238093



In [ ]:
LSTM_AE.save('/home/akozhevnikov/autoencoder.h5')